In [ ]:
import os
import re
import json
import glob
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from itertools import permutations 
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
train_files = glob.glob("../input/coleridgeinitiative-show-us-the-data/train/*.json")
test_files = glob.glob("../input/coleridgeinitiative-show-us-the-data/test/*.json")
submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv', index_col=0)

In [ ]:
df_train_publications = pd.DataFrame()

for train_file in tqdm(train_files):
    file_data = pd.read_json(train_file)
    file_data.insert(0,'pub_id', train_file.split('/')[-1].split('.')[0])
    df_train_publications = pd.concat([df_train_publications, file_data])    
    
# Generate the testing publications dataframe
df_test_publications = pd.DataFrame()

for test_file in test_files:
    file_data = pd.read_json(test_file)
    file_data.insert(0,'pub_id', test_file.split('/')[-1].split('.')[0])
    df_test_publications = pd.concat([df_test_publications, file_data])

df_test_publications.to_csv("df_test_publications.csv",index=False)
df_train_publications.to_csv("df_train_publications.csv",index=False)


def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
# error submission

# def filter_jacob(arg: list)-> str:         
#     if len(arg) > 1: 
#         perm = permutations(range(len(arg)))
#         tmp_w = []
#         tmp_s = []
#         for i in perm:
#             tmp_w.append(arg[i[0]]+ '|' +arg[i[1]])
#             tmp_s.append(jaccard(arg[i[0]], arg[i[1]]))
#         out = tmp_w[np.argmax(tmp_s)]      
#     else:
#         out = arg[0]
#     return out

# sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
# titles = [x.lower() for x in set(train['dataset_title'].unique()).union(set(train['dataset_label'].unique()))]

# for id_ in sub.Id.values:
#     txt = df_test_publications[df_test_publications.pub_id == id_].text.str.cat(sep = '\n').lower()
#     label = []
#     for tit in titles:
#         if tit in txt:
#             label.append(clean_text(tit))    
#     txt = filter_jacob(label)
#     print(txt)
#     sub.loc[sub.Id == id_,'PredictionString'] = txt
    
# sub.to_csv('submission.csv')
# sub

In [ ]:
submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv', index_col=0)
datasets_titles = [x.lower() for x in set(train['dataset_title'].unique()).union(set(train['dataset_label'].unique()))]

labels = []
for index in submission_df.index:
    publication_text = df_test_publications[df_test_publications['pub_id'] == index].text.str.cat(sep='\n').lower()
    label = []
    for dataset_title in datasets_titles:
        if dataset_title in publication_text:
            label.append(clean_text(dataset_title))
    labels.append('|'.join(label))

submission_df['PredictionString'] = labels

submission_df.to_csv('submission.csv')

submission_df